## Knowledge representation and Learning

### Minesweeper project

#### Alvise Dei Rossi #2004250

In [1]:
import numpy as np
import random
from pysat.card import *
import pysat
from pysat.solvers import Minisat22
from time import perf_counter
from copy import deepcopy
from scipy.special import comb
from scipy import median
import pprint
import warnings

In [2]:
warnings.filterwarnings("ignore")

##### Helper functions to setup the board

In [3]:
def map_index_to_integer(i,j,size):
    ''' Helper function that returns integer corresponding to a square of the board'''
    return i*size+j

def map_integer_to_index(integer,size):
    '''Helper function that returns the index of the square corresponding to an int'''
    return (integer//size,integer%size)

def adjacent_indexes(i,j,size):
    '''Helper function that returns a set of the adjacent indexes of a square, considering size and borders'''
    assert 0 <= i < size
    assert 0 <= j < size
    square = set([(max(i-1,0),max(j-1,0)),(max(i-1,0),j),(max(i-1,0),min(j+1,size-1)),
                (i,max(j-1,0)),(i,j),(i,min(j+1,size-1)),
                (min(i+1,size-1),max(j-1,0)),(min(i+1,size-1),j),(min(i+1,size-1),min(j+1,size-1))])
    square.remove((i,j))
    return square

In [4]:
def setup_board(size, n_mines, firstclick=(0,0),initial_click_0=True):
    '''Sets up the board at the beginning of the game'''
    board = np.zeros((size,size),dtype=np.int8)
    i,j = firstclick
    places = set() # determine where to put mines
    if initial_click_0: # in some versions of minesweeper the firstclick is always a 0, hence no mine can be around it
        impossible_indexes = adjacent_indexes(i,j,size)
        impossible_indexes.add((i,j))
        impossible_placements = [map_index_to_integer(i,j,size) for i,j in impossible_indexes]

        while len(places) < n_mines:
            place = random.randint(0,size*size-1)
            if place not in impossible_placements:
                places.add(place)
    else: # if we don't want the rule that the first click must be on a 0 square
        while len(places) < n_mines:
            place = random.randint(0,size*size-1)
            if place != map_index_to_integer(*firstclick,size):
                places.add(place)
            
    mines_indexes = [map_integer_to_index(place,size) for place in places] # convert int placements in tuples (i,j) of coordinates
    for index in mines_indexes: # set up the mines
        board[index] = 9 # every mine is encoded into integer 9 in the board
    for i in range(size): # count adjacent mines for every square of the board and set the value of the board to it
        for j in range(size):
            if board[i,j] != 9:
                board[i,j] = list(board[max(i-1,0):min(i+2,size),max(j-1,0):min(j+2,size)].reshape(-1)).count(9)
    return board, set(mines_indexes)

In [5]:
def print_board(board,only_revealed_squares=False,unrevealed_squares=[],mines=[]):
    '''Prints the board to the screen'''
    printboard = board.astype(str)
    printboard[printboard=="9"] = "M"
    idxs = np.array([str(i%10) for i in range(len(board))])
    print("   ",idxs,"\n")
    if only_revealed_squares:
        for index in unrevealed_squares:
            i,j = index
            printboard[i,j] = "X" # hide the parts of the grid still unknown
        for index in mines:
            i,j = index
            printboard[i,j] = "F" # flag the known mines
    for i,row in enumerate(printboard):
        print(i%10," ", row)
    #print(printboard)

In [6]:
# example of board and its printing
random.seed(42)

size = 8
n_mines = 10
unrevealed_squares = set([(i,j) for i in range(size) for j in range(size)])
firstclick = (random.randint(0,size-1),random.randint(0,size-1))
board, mines_indexes = setup_board(size,n_mines,firstclick,True)
print_board(board,False,unrevealed_squares,[])

    ['0' '1' '2' '3' '4' '5' '6' '7'] 

0   ['0' '0' '2' 'M' 'M' '2' '1' '0']
1   ['0' '0' '2' 'M' '4' 'M' '1' '0']
2   ['0' '0' '2' '3' '5' '3' '3' '1']
3   ['0' '0' '2' 'M' 'M' 'M' '2' 'M']
4   ['0' '0' '2' 'M' '4' '2' '2' '1']
5   ['0' '0' '1' '1' '1' '1' '1' '1']
6   ['0' '0' '0' '0' '0' '1' 'M' '1']
7   ['0' '0' '0' '0' '0' '1' '1' '1']


#### Problem encoding

We'll use variables $x_{ij}$ to indicate if in (i,j) there's a mine ($x_{ij}$ = True) or not ($x_{ij}$ = False)

In [7]:
def x(i,j,size):
    '''encode the presence of a mine in (i,j) into an integer'''
    return map_index_to_integer(i,j,size)+1

In [8]:
for i in range(size):
    for j in range(size):
        print(f"Position {i,j}, integer: {x(i,j,size)}")

Position (0, 0), integer: 1
Position (0, 1), integer: 2
Position (0, 2), integer: 3
Position (0, 3), integer: 4
Position (0, 4), integer: 5
Position (0, 5), integer: 6
Position (0, 6), integer: 7
Position (0, 7), integer: 8
Position (1, 0), integer: 9
Position (1, 1), integer: 10
Position (1, 2), integer: 11
Position (1, 3), integer: 12
Position (1, 4), integer: 13
Position (1, 5), integer: 14
Position (1, 6), integer: 15
Position (1, 7), integer: 16
Position (2, 0), integer: 17
Position (2, 1), integer: 18
Position (2, 2), integer: 19
Position (2, 3), integer: 20
Position (2, 4), integer: 21
Position (2, 5), integer: 22
Position (2, 6), integer: 23
Position (2, 7), integer: 24
Position (3, 0), integer: 25
Position (3, 1), integer: 26
Position (3, 2), integer: 27
Position (3, 3), integer: 28
Position (3, 4), integer: 29
Position (3, 5), integer: 30
Position (3, 6), integer: 31
Position (3, 7), integer: 32
Position (4, 0), integer: 33
Position (4, 1), integer: 34
Position (4, 2), intege

Initially we know nothing of the board, except the global constraint that in the whole board there's a precise number of mines (n_mines).

When we reveal squares, if it's a mine, the game is lost, otherwise their information is encoded into cardinality constraints. For example if in index (2,2) the board is showing 3, we know that between its adjacent squares there are exactly three mines.

Equality cardinality constraints of the type "exactly k adjacent cells to x(i,j) contain mines" can be expressed as such:


$$\bigwedge_{I\subseteq{neighbors(x_{ij})},|I| = n-k+1}\bigvee_{x_{i'j'} \in I} x_{i',j'} \wedge \bigwedge_{I\subseteq{neighbors(x_{ij})},|I| = k+1}\bigvee_{x_{i'j'} \in I} \neg x_{i',j'} $$

with n being the cardinality of the set of the neighbors of x(i,j).

In [9]:
def findsubsets(s, n):
    import itertools
    return list(itertools.combinations(s, n))

In [10]:
# example

neighbors = set([x(i,j,size) for i,j in adjacent_indexes(3,2,size)])
k = board[3,2]

for subset in findsubsets(neighbors,len(neighbors)-k+1):
    clause = [item for item in subset]
    print("adding",clause)

print("\n")

for subset in findsubsets(neighbors,k+1):
    clause = [-item for item in subset]
    print("adding",clause)

adding [34, 35, 36, 18, 19, 20, 26]
adding [34, 35, 36, 18, 19, 20, 28]
adding [34, 35, 36, 18, 19, 26, 28]
adding [34, 35, 36, 18, 20, 26, 28]
adding [34, 35, 36, 19, 20, 26, 28]
adding [34, 35, 18, 19, 20, 26, 28]
adding [34, 36, 18, 19, 20, 26, 28]
adding [35, 36, 18, 19, 20, 26, 28]


adding [-34, -35, -36]
adding [-34, -35, -18]
adding [-34, -35, -19]
adding [-34, -35, -20]
adding [-34, -35, -26]
adding [-34, -35, -28]
adding [-34, -36, -18]
adding [-34, -36, -19]
adding [-34, -36, -20]
adding [-34, -36, -26]
adding [-34, -36, -28]
adding [-34, -18, -19]
adding [-34, -18, -20]
adding [-34, -18, -26]
adding [-34, -18, -28]
adding [-34, -19, -20]
adding [-34, -19, -26]
adding [-34, -19, -28]
adding [-34, -20, -26]
adding [-34, -20, -28]
adding [-34, -26, -28]
adding [-35, -36, -18]
adding [-35, -36, -19]
adding [-35, -36, -20]
adding [-35, -36, -26]
adding [-35, -36, -28]
adding [-35, -18, -19]
adding [-35, -18, -20]
adding [-35, -18, -26]
adding [-35, -18, -28]
adding [-35, -19, -2

#### Functions to run the game

In [11]:
def update_game(MS,index,size,type_of_encoding="Sinz"):
    '''Add clauses relevant to the square which has been selected, returns a boolean saying if the game is lost'''
    global clauses
    if board[index] == 9: # if we click on a mine, the game is over
        unrevealed_squares.discard(index) # this is just to allow to show the mine clicked in the printed board
        return True # game is lost
    else:
        clause = [-x(*index,size)]
        MS.add_clause(clause) # add clause that there's no mine in that square
        clauses += [clause]
    adj_squares = adjacent_indexes(*index,size) # consider all squares adjacent to the chosen square
    literals = [x(i,j,size) for i,j in adj_squares] # consider the literals of the adjacent squares
    k = board[index] # k of them contain mines
    unrevealed_squares.discard(index) # the square selected is now visited and its information available
    relevant_squares.update(adj_squares.intersection(unrevealed_squares.difference(mines))) # adjacent unrevealed squares are relevant
    relevant_squares.discard(index) # since we now know all about the selected square, it's no longer relevant
    safe_visited.add(index) # add it to the visited and safe squares
    if type_of_encoding=="Sinz": # use predifined efficient encodings of pysat
        cardinality_clauses = CardEnc.equals(lits=literals,bound=k,top_id=MS.nof_vars()) # encoded cardinality constraints
        MS.append_formula(cardinality_clauses.clauses) # add them to the sat solver
        clauses += cardinality_clauses.clauses
    elif type_of_encoding=="naive": # to use naive way of encoding cardinality constraints
        for subset in findsubsets(literals,len(literals)-k+1): # at least k
            clause = [item for item in subset]
            MS.add_clause(clause)
            clauses.append(clause)
        for subset in findsubsets(literals,k+1): # at most k
            clause = [-item for item in subset]
            MS.add_clause(clause)
            clauses.append(clause)
    else:
        raise Exception("the type of cardinality encoding specified isn't known")
    return False # game is not lost 

In [12]:
def check_relevant_squares(MS,relevant_squares,mines,size):
    """"given the relevant square we want to know if it's safe to reveal them or not"""
    global clauses
    
    safe_squares = set()
    unsure_squares = set()
    for square in relevant_squares: # we consider squares relevant, meaning close to the discovery region
        literal = x(*square,size) # consider the encoding of the square
        with Minisat22(bootstrap_with=clauses) as solver:
            if not solver.solve(assumptions=[literal]): # if the clauses added plus the literal is unsat
                safe_squares.add(square) # then the negation of the literal (i.e. no mine) is a logical consequence
            elif not solver.solve(assumptions=[-literal]): # if the clauses added plus the negation of the literal is unsat
                mines.add(square) # then there's a mine in the square as a logical consequence
                clause = [x(*square,size)]
                MS.add_clause(clause) # add clause that there is a mine in that square
                clauses += [clause]
                unrevealed_squares.discard(square) # flags the mine, since it's logical to say it is one
            else:
                unsure_squares.add(square) # otherwise we can't yet tell if there's a mine or not in that square
    relevant_squares.difference_update(safe_squares) # we eliminate all safe choices from the relevant squares
    relevant_squares.difference_update(mines) # and so we do with all mines, we only leave undecided squares
    is_stuck = (safe_squares == set()) # if no safe action is found, the agent is stuck
    return safe_squares,unsure_squares,is_stuck

In [13]:
def consider_not_adjacent(MS,size):
    '''When the agent gets stucked, before considering model counting, sometimes the solution can be obtained by
    considering some squares which are not adjacent and not relevant'''
    
    relevant_squares.update(unconsidered_squares)
    unconsidered_safe, unconsidered_unsure, is_still_stucked = check_relevant_squares(MS,relevant_squares,mines,size)
    safe_squares.update(unconsidered_safe)
    return is_still_stucked

In [15]:
def add_global_cardinality_constraint(MS,n_mines,type_of_encoding="naive",verbose=False):
    '''Adds constraint in the unrevealed cells there are a number of mines left
    It can be necessary in the endgame, as cells adjacent to only mines would not be considered otherwise'''
    
    global clauses
    
    mines_left = n_mines - len(mines)
    
    if type_of_encoding == "Sinz":
        global_cardinality = CardEnc.equals(lits=list(range(1,size*size+1)),bound=n_mines,encoding=1)
        MS.append_formula(global_cardinality.clauses)
        clauses += global_cardinality.clauses
    elif type_of_encoding == "naive":
        literals = [x(i,j,size) for i,j in unrevealed_squares]
        for subset in findsubsets(literals,len(literals)-mines_left+1): # at least k
            clause = [item for item in subset]
            MS.add_clause(clause)
            clauses.append(clause)
        for subset in findsubsets(literals,mines_left+1): # at most k
            clause = [-item for item in subset]
            MS.add_clause(clause)
            clauses.append(clause)
    else:
        raise Exception("Type of encoding specified not known")
    
    if verbose:
        print("Added global cardinality constraints")
    return True

In [18]:
def decide_safest_square(size,allow_random=True,verbose=False,random_penalization=0.03):
    '''Uses probabilistic reasoning and model enumeration for a reduced minesweeper problem to figure out
    which is the safest square to probe and returns it'''
    import itertools

    global clauses
    
    # consider only cells which are included in clauses added so far
    relevant_cells = set([abs(literal) for clause in clauses for literal in clause])
    # of these cells, we want to calculate the probability of being mines of the unprobed cells
    unrevealed_cells = set([x(i,j,size) for i,j in unrevealed_squares])
    unrevealed_relevant_cells = unrevealed_cells.intersection(relevant_cells)
    # we're cutting off a part of the grid, we need to consider how large is the rest for combinatorial calculations later
    unexplored_area_nof_cells = size*size - len(relevant_cells)

    # we need to instantiate a minisat which uses only the variables seen so far in clauses
    # to do so we need to map the integers between the larger minisat and the smaller one
    relevant_dict = {v:k+1 for k,v in enumerate(relevant_cells)} # positive occurences of literals, re-ordered
    relevant_dict.update({-k:-v for k,v in relevant_dict.items()}) # negative occurences of those literals
    inverse_relevant_dict = {v:k for k,v in relevant_dict.items()} # needed to translate models back to the original problem

    # translate clauses of the original problem in the encoding of the new reduced minisat problem
    m2_clauses = [[relevant_dict.get(literal) for literal in clause] for clause in clauses]

    models = [] # we keep track of the possible models for the reduced problem
    total_models = 0 # normalizing factor when calculating probabilities
    random_models = 0 # # it's best to consider also the approximate probability that a random non-relevant cell is a mine

    with Minisat22(bootstrap_with=m2_clauses) as ms: # this is the reduced problem instantiation
        # in most cases we can enumerate all the models, but rarely we must cut off the process
        top100 = itertools.islice(ms.enum_models(), 100)
        for model in top100:
            # and translate them to the original problem encoding for only the variables encountered so far
            inverse_model = [inverse_relevant_dict[literal] for literal in model]
            model_mines = sum(np.array(inverse_model)>0) # every model can have a different number of mines
            rest_of_mines = n_mines - model_mines # the area not explored will have the rest
            # we compute the possible combinations in which the other mines can be distributed, assuming uniformity
            approx_nof_models_rest_of_mines = comb(unexplored_area_nof_cells,rest_of_mines) 
            random_models += rest_of_mines/unexplored_area_nof_cells * approx_nof_models_rest_of_mines
            total_models += approx_nof_models_rest_of_mines
            models.append((model_mines,inverse_model,approx_nof_models_rest_of_mines)) # save the model and info

    probability_dict = {} # Build a dictionary containing the probability to have a mine for every relevant cell

    for cell in unrevealed_relevant_cells:
        sum_cell_positive_models = 0
        for model in models:
            if cell in model[1]:
                sum_cell_positive_models += model[2] # we sum the number of models for which that literal is positive
        probability_dict[cell] = round(sum_cell_positive_models/total_models,5) # normalize to calculate probability
    
    insight_dict = {map_integer_to_index(k-1,size):v for k,v in probability_dict.items()} # this allows for easier reading of next print
    
    if unrevealed_squares.difference(relevant_squares) != set() and allow_random: # if there are unrelevant cells
    # we compute the probability of a random action, penalize it slightly as usually exploring close is better
        probability_dict[0] = round(random_models/total_models + random_penalization,5) 
        insight_dict["random"] = probability_dict[0]
    
    if verbose:
        print("Probabilities:")
        pprint.pprint(insight_dict) # uncomment to visualize the probabilities calculated
    
    safest_integer = min(probability_dict,key=lambda x:probability_dict[x]) # select the one with the least probability
    safest_cell = map_integer_to_index(safest_integer-1,size) # the safest cell to probe
    better_go_random = safest_integer == 0 # bool which specify if the safest cell is a random unrelevant one
    
    #print("\nTAKING A RISK!") # uncomment if you want to see a note when choosing a square which isn't 100% safe
    
    return safest_integer,safest_cell,better_go_random

In [14]:
def print_outcome(is_won,is_lost,is_stuck):
    '''Displays result of the game'''
    print("\n")
    if is_lost:
        print("LOSE")
    elif is_won:
        print("WIN")
    elif is_stuck:
        print("STUCK")

## Game specs and run the game

In [66]:
########################################################
########################################################

# MODIFY BELOW TO PLAY GAMES

########################################################
########################################################

In [28]:
###############################################################
# Game specs
###############################################################

size = 16 # the grid will be a square of dimensions size x size
n_mines = 40 # the number of the mines all over the grid

firstclick = (random.randint(0,size-1),random.randint(0,size-1)) # the first-click is always safe and chosen randomly
#firstclick = (choose_row,choose_column) # in case we want instead to define the initial square ourselves

# to keep track of the state of the grid
unrevealed_squares = set([(i,j) for i in range(size) for j in range(size)]) # which squares are  not visited
relevant_squares = set() # which are the most relevant, based on adjacency to chosen squares
mines = set() # squares containing mines
safe_visited = set() # squares which are safe and have been visited already

clauses = [] # keep track of added clauses, needed for copies of the sat solver

# boolean vars to check if the game is finished and its outcome
is_lost = False
is_won = False
is_stuck = False

guess_when_stucked = True # decide if we want to try to guess when it gets stucked or not
allow_random = True # and if we allow to choose randomly in an explored region of the grid, when it's better

start_with_0 = False #decide if the first-click must be a 0 or not

type_of_encoding="naive" # determine type of cardinality encodings, probabilistic reasoning must use naive constaints

verbose = True # show additional information like squares deemed safe at every iteration

verbose_probabilistic_reasoning = True # when a not safe move is taken, show the probabilities calculated

added_global = False # flag keeping track if global cardinality constraints have already been imposed

random_penalization = 0.05 # penalizes exploration of random areas when using probabilistic reasoning


##############################################################
# RUN THE GAME
##############################################################


print(f"First-click: {firstclick}\n")

# setup the board and print it, visible
board, mines_indexes = setup_board(size,n_mines,firstclick,start_with_0)
safe_indexes = unrevealed_squares.difference(mines_indexes) # safe squares for exit condition
print("UNCOVERED BOARD CONFIG\n")
print_board(board,False,unrevealed_squares,mines)

# initiate sat solver 
MS = Minisat22()

# initiate the game, the first click is always a safe click
print("\n\nGAME BEGINS\n")
safe_squares = set([firstclick])

while(not is_won and not is_lost and not is_stuck):
    
    if verbose:
        print(f"\nSquares to unconver are: {safe_squares}\n\n") # based on the previous config some squares are safe
        
    for safe_square in safe_squares: # we reaveal each one of the safe squares
        is_lost = update_game(MS,
                              safe_square,
                              size,
                              type_of_encoding) # and encode their information with cardinality constraints
        if is_lost:
            break
            
    print_board(board,True,unrevealed_squares,mines) # print the current status of the board
    #print(f"MINES:{mines}") # uncomment to see cells that are for sure mines so far
    #print(f"RELEVANT:{relevant_squares}") # uncomment to see relevant squares in current situation
    
    if is_lost:
        break
        
    safe_squares, unsure_squares, is_stuck = check_relevant_squares(MS,
                                                                      relevant_squares,
                                                                      mines,size) # find logical consequences
    
    if safe_visited == safe_indexes: # if all safe cells have been probed, the game is won
        is_won = True # the game is won
        break
        
    if is_stuck: # if there are no safe squares to choose
        unconsidered_squares = unrevealed_squares.difference(mines).difference(relevant_squares) 
        if len(unconsidered_squares) <= 10: # if there are few unconsidered squares
            if not added_global: # in the endgame it may be necessary to add global cardinality constraints
                added_global = add_global_cardinality_constraint(MS,n_mines,type_of_encoding,verbose)
            is_stuck = consider_not_adjacent(MS,size) # try to consider them too, to see if it unstucks the agent
        if guess_when_stucked: # if we allow the use of probabilistic reasoning
            if is_stuck: # and if the agent is still stuck
                safest_integer, safest_cell, better_random = decide_safest_square(size,
                                                                                  allow_random,
                                                                                  verbose_probabilistic_reasoning,
                                                                                  random_penalization)
                if better_random:
                    print("\nchoosing randomly")
                    safest_cell = random.sample(list(unrevealed_squares.difference(relevant_squares)),1)[0]
                safe_squares.add(safest_cell)
                is_stuck = False

print_outcome(is_won,is_lost,is_stuck)
if unsure_squares != set() and verbose: # prints out in case it's lost and we want to see which were the unsure cells
    print(unsure_squares)

First-click: (6, 14)

UNCOVERED BOARD CONFIG

    ['0' '1' '2' '3' '4' '5' '6' '7' '8' '9' '0' '1' '2' '3' '4' '5'] 

0   ['0' '0' '0' '0' '1' '2' 'M' '1' '0' '0' '0' '0' '1' 'M' '1' '0']
1   ['0' '0' '1' '1' '2' 'M' '3' '2' '0' '0' '0' '1' '2' '2' '1' '0']
2   ['0' '0' '1' 'M' '3' '4' 'M' '3' '1' '1' '1' '2' 'M' '2' '1' '0']
3   ['0' '0' '1' '2' 'M' '3' 'M' 'M' '1' '1' 'M' '2' '2' 'M' '2' '1']
4   ['0' '0' '1' '2' '3' '3' '3' '2' '1' '2' '2' '2' '1' '2' 'M' '1']
5   ['0' '0' '1' 'M' '3' 'M' '2' '0' '0' '1' 'M' '1' '0' '1' '1' '1']
6   ['0' '0' '1' '1' '3' 'M' '2' '0' '0' '1' '1' '2' '2' '2' '1' '0']
7   ['0' '0' '1' '1' '2' '1' '1' '0' '1' '2' '2' '2' 'M' 'M' '1' '0']
8   ['0' '0' '1' 'M' '1' '0' '0' '0' '1' 'M' 'M' '3' '2' '2' '2' '1']
9   ['0' '0' '1' '1' '2' '1' '1' '0' '1' '3' 'M' '2' '0' '1' '2' 'M']
0   ['1' '1' '1' '0' '1' 'M' '1' '0' '1' '2' '2' '2' '2' '3' 'M' '2']
1   ['2' 'M' '1' '0' '1' '1' '1' '0' '1' 'M' '2' '2' 'M' 'M' '2' '1']
2   ['M' '2' '1' '0' '0' '0' '1' '1' '2' '

 (10, 14): 0.14034}

Squares to unconver are: {(5, 11)}


    ['0' '1' '2' '3' '4' '5' '6' '7' '8' '9' '0' '1' '2' '3' '4' '5'] 

0   ['X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X']
1   ['X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X']
2   ['X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X']
3   ['X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' '2' 'X' 'X' 'X' 'X']
4   ['X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' '2' '1' '2' 'X' 'X']
5   ['X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' '1' 'X' '1' '1' '1']
6   ['X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' '2' '1' '0']
7   ['X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'F' '1' '0']
8   ['X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' '2' '2' '1']
9   ['X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' '1' 'X' 'X']
0   ['X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X']
1   ['X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X']
2   ['X' 'X' 'X' 'X' 'X' 'X' '


Squares to unconver are: {(9, 8), (1, 15), (9, 4), (1, 11), (12, 13), (1, 14), (2, 13), (1, 10), (9, 7), (12, 14), (4, 5), (1, 13), (7, 5), (1, 9), (9, 6), (3, 5), (2, 7), (12, 15), (3, 12), (2, 11), (9, 9), (3, 8), (7, 4), (1, 8), (1, 7), (9, 5), (8, 4)}


    ['0' '1' '2' '3' '4' '5' '6' '7' '8' '9' '0' '1' '2' '3' '4' '5'] 

0   ['X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X']
1   ['X' 'X' 'X' 'X' 'X' 'X' 'X' '2' '0' '0' '0' '1' 'X' '2' '1' '0']
2   ['X' 'X' 'X' 'X' 'X' 'X' 'X' '3' '1' '1' '1' '2' 'F' '2' '1' '0']
3   ['X' 'X' 'X' 'X' 'X' '3' 'F' 'F' '1' '1' 'F' '2' '2' 'F' '2' '1']
4   ['X' 'X' 'X' 'X' 'X' '3' '3' '2' '1' '2' '2' '2' '1' '2' 'F' '1']
5   ['X' 'X' 'X' 'X' 'X' 'F' '2' '0' '0' '1' 'F' '1' '0' '1' '1' '1']
6   ['X' 'X' 'X' 'X' 'X' 'F' '2' '0' '0' '1' '1' '2' '2' '2' '1' '0']
7   ['X' 'X' 'X' 'X' '2' '1' '1' '0' '1' '2' '2' '2' 'F' 'F' '1' '0']
8   ['X' 'X' 'X' 'X' '1' '0' '0' '0' '1' 'F' 'F' '3' '2' '2' '2' '1']
9   ['X' 'X' 'X' 'X' '2' '1' '1' '0' '1


Squares to unconver are: {(0, 1), (0, 0)}


    ['0' '1' '2' '3' '4' '5' '6' '7' '8' '9' '0' '1' '2' '3' '4' '5'] 

0   ['0' '0' '0' '0' '1' '2' 'F' '1' '0' '0' '0' '0' '1' 'F' '1' '0']
1   ['0' '0' '1' '1' '2' 'F' '3' '2' '0' '0' '0' '1' '2' '2' '1' '0']
2   ['0' '0' '1' 'F' '3' '4' 'F' '3' '1' '1' '1' '2' 'F' '2' '1' '0']
3   ['0' '0' '1' '2' 'F' '3' 'F' 'F' '1' '1' 'F' '2' '2' 'F' '2' '1']
4   ['0' '0' '1' '2' '3' '3' '3' '2' '1' '2' '2' '2' '1' '2' 'F' '1']
5   ['0' '0' '1' 'F' '3' 'F' '2' '0' '0' '1' 'F' '1' '0' '1' '1' '1']
6   ['0' '0' '1' '1' '3' 'F' '2' '0' '0' '1' '1' '2' '2' '2' '1' '0']
7   ['0' '0' '1' '1' '2' '1' '1' '0' '1' '2' '2' '2' 'F' 'F' '1' '0']
8   ['0' '0' '1' 'F' '1' '0' '0' '0' '1' 'F' 'F' '3' '2' '2' '2' '1']
9   ['0' '0' '1' '1' '2' '1' '1' '0' '1' '3' 'F' '2' '0' '1' '2' 'F']
0   ['1' '1' '1' '0' '1' 'F' '1' '0' '1' '2' '2' '2' '2' '3' 'F' '2']
1   ['2' 'F' '1' '0' '1' '1' '1' '0' '1' 'F' '2' '2' 'F' 'F' '2' '1']
2   ['F' '2' '1' '0' '0' '0' '1' '1' '2' '1

In [29]:
# The solution is unique if we won:
if is_won:
    print(MS.solve()) # there's a solution
    m = MS.get_model() # this is the unique model
    MS.add_clause([-i for i in m]) # if I impose the clause that something has to be different from the solution model
    print(MS.solve()) # the problem is no longer satisfiable

True
False


#### Performance study

In [ ]:
specs = [(4,5),(6,6),(8,10),(16,40),(22,99)] # (size, n_mines) i.e. n_mines in a size x size grid 
start_with_0s = [False, True]
guesses = [False, True]

number_of_tests = 100

times = np.zeros((len(specs),2,2,number_of_tests))
wins = np.zeros((len(specs),2,2))
stucks = np.zeros((len(specs),2,2))
losses = np.zeros((len(specs),2,2))

for i in range(len(specs)):
    for j in range(2):
        for k in range(2):
            for z in range(number_of_tests):
                
                t0 = perf_counter()
                
                ###############################################################
                # Game specs
                ###############################################################

                size = specs[i][0] # the grid will be a square of dimensions size x size
                n_mines = specs[i][1] # the number of the mines all over the grid

                firstclick = (random.randint(0,size-1),random.randint(0,size-1)) # the first-click is always safe and chosen randomly
                #firstclick = (choose_row,choose_column) # in case we want instead to define the initial square ourselves

                # to keep track of the state of the grid
                unrevealed_squares = set([(i,j) for i in range(size) for j in range(size)]) # which squares are  not visited
                relevant_squares = set() # which are the most relevant, based on adjacency to chosen squares
                mines = set() # squares containing mines
                safe_visited = set() # squares which are safe and have been visited already

                clauses = [] # keep track of added clauses, needed for copies of the sat solver

                # boolean vars to check if the game is finished and its outcome
                is_lost = False
                is_won = False
                is_stuck = False

                guess_when_stucked = guesses[k] # decide if we want to try to guess when it gets stucked or not
                allow_random = True # and if we allow to choose randomly when it's better

                start_with_0 = start_with_0s[j] #decide if the first-click must be a 0 or not

                type_of_encoding="naive" # determine type of cardinality encodings, probabilistic reasoning must use naive constaints

                verbose = False # show additional information like squares deemed safe at every iteration

                verbose_probabilistic_reasoning = False # when a not safe move is taken, show the probabilities calculated

                added_global = False # flag keeping track if global cardinality constraints have already been imposed
                
                random_penalization = 0.07 # strong penality but not too strong when 3-4s are found initially


                ##############################################################
                # RUN THE GAME
                ##############################################################


                #print(f"First-click: {firstclick}\n")

                # setup the board and print it, visible
                board, mines_indexes = setup_board(size,n_mines,firstclick,start_with_0)
                safe_indexes = unrevealed_squares.difference(mines_indexes) # safe squares for exit condition
                #print("UNCOVERED BOARD CONFIG\n")
                #print_board(board,False,unrevealed_squares,mines)

                # initiate sat solver 
                MS = Minisat22()

                # initiate the game, the first click is always a safe click
                #print("\n\nGAME BEGINS\n")
                safe_squares = set([firstclick])

                while(not is_won and not is_lost and not is_stuck):

                    if verbose:
                        print(f"\nSquares to unconver are: {safe_squares}\n\n") # based on the previous config some squares are safe

                    for safe_square in safe_squares: # we reaveal each one of the safe squares
                        is_lost = update_game(MS,
                                              safe_square,
                                              size,
                                              type_of_encoding) # and encode their information with cardinality constraints
                        if is_lost:
                            break

                    #print_board(board,True,unrevealed_squares,mines) # print the current status of the board
                    #print(f"MINES:{mines}") # uncomment to see cells that are for sure mines so far
                    #print(f"RELEVANT:{relevant_squares}") # uncomment to see relevant squares in current situation

                    if is_lost:
                        break

                    safe_squares, unsure_squares, is_stuck = check_relevant_squares(MS,
                                                                                      relevant_squares,
                                                                                      mines,size) # find logical consequences

                    if safe_visited == safe_indexes: # if all safe cells have been probed, the game is won
                        is_won = True # then the game is won
                        break

                    if is_stuck: # if there are no safe squares to choose
                        unconsidered_squares = unrevealed_squares.difference(mines).difference(relevant_squares) 
                        if len(unconsidered_squares) <= 5: # if there are few unconsidered squares
                            if not added_global: # in the endgame it may be necessary to add global cardinality constraints
                                added_global = add_global_cardinality_constraint(MS,n_mines,type_of_encoding,verbose)
                            is_stuck = consider_not_adjacent(MS,size) # try to consider them too, to see if it unstucks the agent
                        if guess_when_stucked:
                            if is_stuck: # if the agent is still stuck
                                safest_integer, safest_cell, better_random = decide_safest_square(size,
                                                                                                  allow_random,
                                                                                                  verbose_probabilistic_reasoning,
                                                                                                  random_penalization)
                                if better_random:
                                    #print("\nchoosing randomly")
                                    safest_cell = random.sample(list(unrevealed_squares.difference(relevant_squares)),1)[0]
                                safe_squares.add(safest_cell)
                                is_stuck = False
                if is_won:
                    wins[i,j,k] +=1
                elif is_stuck:
                    stucks[i,j,k] += 1
                elif is_lost:
                    losses[i,j,k] += 1
                #print_outcome(is_won,is_lost,is_stuck)
                if unsure_squares != set() and verbose: # prints out in case it's lost and we want to see which were the unsure cells
                    print(unsure_squares)
                t1 = perf_counter()
                times[i,j,k,z] = (t1-t0)

In [65]:
for i in range(len(specs)):
    print(f"size: {specs[i][0]}x{specs[i][0]}")
    print(f"number of mines: {specs[i][1]}\n")
    for k in range(2):
        for j in range(2):
            print(f"Starts with 0: {start_with_0s[j]}")
            print(f"Uses probabilistic reasoning and #sat {guesses[k]}")
            print(f"Mean time: {round(times[i,j,k,:].mean(),3)}")
            print(f"Median time: {round(median(times[i,j,k,:]),3)}")
            print(f"Win percentage: {wins[i,j,k]/number_of_tests}\n")
    print("\n\n############################################\n\n")

size: 4x4
number of mines: 5

Starts with 0: False
Uses probabilistic reasoning and #sat False
Mean time: 0.002
Median time: 0.001
Win percentage: 0.08

Starts with 0: True
Uses probabilistic reasoning and #sat False
Mean time: 0.007
Median time: 0.006
Win percentage: 0.55

Starts with 0: False
Uses probabilistic reasoning and #sat True
Mean time: 0.148
Median time: 0.014
Win percentage: 0.52

Starts with 0: True
Uses probabilistic reasoning and #sat True
Mean time: 0.014
Median time: 0.008
Win percentage: 0.77



############################################


size: 6x6
number of mines: 6

Starts with 0: False
Uses probabilistic reasoning and #sat False
Mean time: 0.006
Median time: 0.002
Win percentage: 0.16

Starts with 0: True
Uses probabilistic reasoning and #sat False
Mean time: 0.032
Median time: 0.019
Win percentage: 0.73

Starts with 0: False
Uses probabilistic reasoning and #sat True
Mean time: 0.032
Median time: 0.024
Win percentage: 0.72

Starts with 0: True
Uses probabilist